In [1]:
include("AZP_env.jl")
include("AZP_agt.jl")
include("AZP_mcts_valMAX.jl")

check_RL (generic function with 1 method)

In [2]:
mutable struct ReplayBuffer
    buffer::Vector{Agent}
    buffer_size::Int
    batch_size::Int
    #count::Int
end

function init_buffer(buffer_size::Int, batch_size::Int)
    return ReplayBuffer([], buffer_size, batch_size)
end


function save_game!(buffer::ReplayBuffer, agt::Agent)
    if length(buffer.buffer) > buffer.buffer_size
        popfirst!(buffer.buffer)
    end
    push!(buffer.buffer, agt)
end

save_game! (generic function with 1 method)

In [3]:
mutable struct Storage
    storage::Dict{Int, Chain}
    random_out::Chain
    scores::Dict{Vector{Int}, Float32}
end

function init_storage(env)
    return Storage(Dict(), Chain(Dense(zeros(Float32, env.output,env.input_dim))), Dict())
end

function latest_model(storage::Storage)
    if(isempty(storage.storage))
        return storage.random_out
    else
        return storage.storage[rand(keys(storage.storage))]
    end
end

latest_model (generic function with 1 method)

In [4]:
tanh10(x) = Float32(10)*tanh(x)
tanh2(x) = Float32(2)*tanh(x)

tanh2 (generic function with 1 method)

In [5]:
env0 = init_Env(["8", "240", "128", "12", "200", "48", "1", "600", "0.3", "0.25", "50", "4", "15.0", "0.4", "1.0", "0.7", "0.5", "120", "0.75", "0.000001"])

max_turn:  8
num_player:  240
middle_dim:  128
depth:  12
training_step:  200
batch_size:  48
batch_num:  1
num_simulation:  600
α:  0.3
frac:  0.25


Env(8, 240, 2, 3, 1, 6, 48, 128, 7, 12, 200, 400, 48, 1, 0.0001f0, 0.9f0, 600, 0.3f0, 0.25f0, 50, 4, 15.0f0, 0.4f0, 1.0f0, 0.7f0, 0.5f0, ComplexF32[-2.0f0 + 0.0f0im 0.0f0 + 0.0f0im 0.0f0 + 0.0f0im -0.7f0 + 0.0f0im; 0.0f0 - 0.0f0im 1.0f0 + 0.0f0im -0.7f0 + 0.0f0im 0.0f0 + 0.0f0im; 0.0f0 - 0.0f0im -0.7f0 - 0.0f0im 1.0f0 + 0.0f0im 0.0f0 + 0.0f0im; -0.7f0 - 0.0f0im 0.0f0 - 0.0f0im 0.0f0 - 0.0f0im 0.0f0 + 0.0f0im], ComplexF32[0.0f0 + 0.0f0im -0.4f0 + 0.0f0im -0.4f0 + 0.0f0im 0.0f0 + 0.0f0im; -0.4f0 - 0.0f0im 0.0f0 + 0.0f0im 0.0f0 + 0.0f0im -0.4f0 + 0.0f0im; -0.4f0 - 0.0f0im 0.0f0 - 0.0f0im 0.0f0 + 0.0f0im -0.4f0 + 0.0f0im; 0.0f0 - 0.0f0im -0.4f0 - 0.0f0im -0.4f0 - 0.0f0im 0.0f0 + 0.0f0im], 0.00837758f0, 120, 0.75f0, 1.0f-6)

In [6]:
storage0 = init_storage(env0)

Storage(Dict{Int64, Chain}(), Chain(Dense(48 => 7)), Dict{Vector{Int64}, Float32}())

In [7]:
replay_buffer = init_buffer(1000, env0.batch_size)

ReplayBuffer(Agent[], 1000, 48)

In [9]:
model = latest_model(storage0) |> gpu

Chain(
  Dense(48 => 7),                       # 343 parameters  (all zero)
) 

In [11]:
game = play_physics!(env0, model, 4.0f0, 1.0f0)

Agent([5, 4, 6, 4, 2, 2, 1, 1], Int64[], Vector{Float32}[[0.195, 0.125, 0.125, 0.14, 0.29, 0.125], [0.15, 0.165, 0.15166667, 0.38333333, 0.0, 0.15], [0.15, 0.24666667, 0.15, 0.0, 0.17166667, 0.28166667], [0.0, 0.225, 0.27, 0.31666666, 0.18833333, 0.0], [0.40166667, 0.59833336, 0.0, 0.0, 0.0, 0.0], [0.37666667, 0.62333333, 0.0, 0.0, 0.0, 0.0], [0.62333333, 0.37666667, 0.0, 0.0, 0.0, 0.0], [0.60333335, 0.39666668, 0.0, 0.0, 0.0, 0.0]])

In [13]:
game.child_visit_pi

8-element Vector{Vector{Float32}}:
 [0.195, 0.125, 0.125, 0.14, 0.29, 0.125]
 [0.15, 0.165, 0.15166667, 0.38333333, 0.0, 0.15]
 [0.15, 0.24666667, 0.15, 0.0, 0.17166667, 0.28166667]
 [0.0, 0.225, 0.27, 0.31666666, 0.18833333, 0.0]
 [0.40166667, 0.59833336, 0.0, 0.0, 0.0, 0.0]
 [0.37666667, 0.62333333, 0.0, 0.0, 0.0, 0.0]
 [0.62333333, 0.37666667, 0.0, 0.0, 0.0, 0.0]
 [0.60333335, 0.39666668, 0.0, 0.0, 0.0, 0.0]

In [12]:
make_target(env0, game, 3)

7-element Vector{Float32}:
 0.15
 0.24666667
 0.15
 0.0
 0.17166667
 0.28166667
 4.636553

In [3]:
using LinearAlgebra

In [7]:
maximum([1.0, 2.0, 3.0])

3.0

In [8]:
minimum([1.0, 2.0, 3.0])

1.0

In [9]:
argmax([1.0, 2.0, 3.0])

3

In [12]:
a = [i for i in 1:10]

10-element Vector{Int64}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10

In [13]:
w = a.^2
w = w/sum(w)

10-element Vector{Float64}:
 0.0025974025974025974
 0.01038961038961039
 0.023376623376623377
 0.04155844155844156
 0.06493506493506493
 0.09350649350649351
 0.12727272727272726
 0.16623376623376623
 0.21038961038961038
 0.2597402597402597

In [17]:
using Distributions
using StatsBase

In [26]:
using BenchmarkTools

In [27]:
@benchmark ind = sample(a, ProbabilityWeights(w), 1)

BenchmarkTools.Trial: 10000 samples with 803 evaluations.
 Range (min … max):  157.186 ns …   3.694 μs  ┊ GC (min … max): 0.00% … 93.55%
 Time  (median):     163.973 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   170.550 ns ± 139.887 ns  ┊ GC (mean ± σ):  3.24% ±  3.77%

          ▂▄▇█▄▁                                                 
  ▁▁▁▂▄▆▇███████▆▄▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  157 ns           Histogram: frequency by time          195 ns <

 Memory estimate: 96 bytes, allocs estimate: 2.

In [28]:
@benchmark ind = rand(DiscreteNonParametric(a, w))

BenchmarkTools.Trial: 10000 samples with 225 evaluations.
 Range (min … max):  334.236 ns …   6.193 μs  ┊ GC (min … max): 0.00% … 92.87%
 Time  (median):     344.742 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   363.250 ns ± 270.864 ns  ┊ GC (mean ± σ):  3.44% ±  4.35%

    ▄▇█▇▅▂▁▂▂▁▁                                 ▁▁▁▁            ▂
  ▆███████████████▆▅▇█▇▆▄▄▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▆▆▇████▇▇▆▆▆▆▅▅▆▆▇ █
  334 ns        Histogram: log(frequency) by time        473 ns <

 Memory estimate: 464 bytes, allocs estimate: 4.

In [1]:
using JLD2
using FileIO

In [2]:
a = Dict()

Dict{Any, Any}()

In [14]:
for i in 1:16
    a[[j for j in 1:i]] = 0.3i
end

In [3]:
a[[1, 2, 3]] = 0.4
a[[1,2]] = 2.0

2.0

In [5]:
b = Dict()
b["x"] = 2.0
b["y"] = 3.0

3.0

In [6]:
save("test.jld2", b)

In [7]:
c = load("test.jld2")

Dict{String, Any} with 2 entries:
  "x" => 2.0
  "y" => 3.0

In [8]:
c["x"]

2.0

In [6]:
function dict_copy(orig::Dict)
    c_dict = Dict{String, Float32}()
    for k in keys(orig)
        c_dict["$(k)"] = Float32(orig[k]) 
    end
    return c_dict
end

dict_copy (generic function with 1 method)

In [4]:
length(a)

2

In [5]:
keys(a)

KeySet for a Dict{Any, Any} with 2 entries. Keys:
  [1, 2, 3]
  [1, 2]

In [7]:
d = dict_copy(a)

Dict{String, Float32} with 2 entries:
  "[1, 2]"    => 2.0
  "[1, 2, 3]" => 0.4

In [8]:
@show d

d = Dict{String, Float32}("[1, 2]" => 2.0, "[1, 2, 3]" => 0.4)


Dict{String, Float32} with 2 entries:
  "[1, 2]"    => 2.0
  "[1, 2, 3]" => 0.4

In [12]:
dict = Dict("a" => 1, "b" => 2, "c" => 3, "d" => 4, "e" => 5, "f" => 5, "g" => 5)

Dict{String, Int64} with 7 entries:
  "f" => 5
  "g" => 5
  "c" => 3
  "e" => 5
  "b" => 2
  "a" => 1
  "d" => 4

In [19]:
maximum(dict)[2]

5

In [24]:
k = [keys(dict)...]

7-element Vector{String}:
 "f"
 "g"
 "c"
 "e"
 "b"
 "a"
 "d"

In [28]:
inds = findall(s -> dict[s] == maximum(dict)[2], [keys(dict)...])

3-element Vector{Int64}:
 1
 2
 4

In [29]:
ind_s = [k[i] for i in inds]

3-element Vector{String}:
 "f"
 "g"
 "e"